<a href="https://colab.research.google.com/github/hasankhwaja/Movie-Sentiment-Analysis-/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Mounted at /content/drive


In [ ]:
link = 'https://drive.google.com/file/d/1l_5o4Bkbs3Uab_WezeCoo2C3JCb8BrBc/view'

import pandas as pd

# to get the id part of the file
id = link.split("/")[-2]
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train.tsv')
train = pd.read_csv('train.tsv', sep = '\t')
#########
link = 'https://drive.google.com/file/d/1ID_g_xHfJ2NkXLIxAnL9m05pCaDHSteH/view'
id = link.split("/")[-2]
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('test.tsv')

test = pd.read_csv('test.tsv', sep = '\t')

In [ ]:
'''

Here is the start of another BERT implementation

'''

import os
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from wordcloud import WordCloud, STOPWORDS



In [ ]:
'''
# Load your datasets
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')
'''

# Preprocessing: Clean text
def text_cleaning(text):
    if not isinstance(text, str):
        return ""
    soup = BeautifulSoup(text, "html.parser")
    text = re.sub(r'\[[^]]*\]', '', soup.get_text())
    pattern = r"[^a-zA-Z0-9\s,']"
    text = re.sub(pattern, '', text)
    return text

# Handle missing or invalid values in 'Phrase' column
train['Phrase'] = train['Phrase'].fillna('')
test['Phrase'] = test['Phrase'].fillna('')

# Apply text cleaning
train['Cleaned_sentence'] = train['Phrase'].apply(text_cleaning)
test['Cleaned_sentence'] = test['Phrase'].apply(text_cleaning)

# Labels
train_targets = train['Sentiment']

# Train-validation split
x_train, x_val, y_train, y_val = train_test_split(
    train['Cleaned_sentence'], train_targets, test_size=0.2, stratify=train_targets)

<ipython-input-7-3198caf006f7>:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
<ipython-input-7-3198caf006f7>:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len = 128

# Tokenize and encode data
X_train_encoded = tokenizer.batch_encode_plus(x_train.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length=max_len,
                                              return_tensors='tf')

X_val_encoded = tokenizer.batch_encode_plus(x_val.tolist(),
                                            padding=True,
                                            truncation=True,
                                            max_length=max_len,
                                            return_tensors='tf')

X_test_encoded = tokenizer.batch_encode_plus(test['Cleaned_sentence'].tolist(),
                                             padding=True,
                                             truncation=True,
                                             max_length=max_len,
                                             return_tensors='tf')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Initialize the BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)  # 5 labels for your dataset

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model
history = model.fit(
    [X_train_encoded['input_ids'], X_train_encoded['attention_mask']],
    y_train,
    validation_data=(
        [X_val_encoded['input_ids'], X_val_encoded['attention_mask']], y_val
    ),
    batch_size=32,
    epochs=2
)

# Save the model and tokenizer
model.save_pretrained('bert_model')
tokenizer.save_pretrained('bert_tokenizer')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
3902/3902 [==============================] - 1991s 499ms/step - loss: 0.7941 - accuracy: 0.6681 - val_loss: 0.7368 - val_accuracy: 0.6872
Epoch 2/2
3902/3902 [==============================] - 1947s 499ms/step - loss: 0.6590 - accuracy: 0.7260 - val_loss: 0.7189 - val_accuracy: 0.6973


('bert_tokenizer/tokenizer_config.json',
 'bert_tokenizer/special_tokens_map.json',
 'bert_tokenizer/vocab.txt',
 'bert_tokenizer/added_tokens.json')

In [ ]:
# Predict on the test set
predictions = model.predict([X_test_encoded['input_ids'], X_test_encoded['attention_mask']])
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

2072/2072 [==============================] - 301s 144ms/step


In [ ]:
# Add predictions to the test dataframe
label_map = {
    0: "Very Negative",
    1: "Negative",
    2: "Neutral",
    3: "Positive",
    4: "Very Positive"
}

test['Predicted_Sentiment'] = [label_map[label] for label in predicted_labels]

# Save the predictions
test[['Phrase', 'Predicted_Sentiment']].to_csv('test_predictions.csv', index=False)

# Display some example predictions
print(test[['Phrase', 'Predicted_Sentiment']].head())
print(test[['Phrase', 'Predicted_Sentiment']].tail())

                                              Phrase Predicted_Sentiment
0  An intermittently pleasing but mostly routine ...             Neutral
1  An intermittently pleasing but mostly routine ...             Neutral
2                                                 An             Neutral
3  intermittently pleasing but mostly routine effort             Neutral
4         intermittently pleasing but mostly routine             Neutral
                                       Phrase Predicted_Sentiment
66287  A long-winded , predictable scenario .            Negative
66288    A long-winded , predictable scenario            Negative
66289                         A long-winded ,            Negative
66290                           A long-winded            Negative
66291                    predictable scenario            Negative


In [ ]:
# Map predicted labels (numerical) back to the sentiment classes if needed
test['Sentiment'] = tf.argmax(predictions.logits, axis=1).numpy()

# Write only PhraseId and Sentiment to a new CSV file
test[['PhraseId', 'Sentiment']].to_csv('test_predictions_phraseid_sentiment.csv', index=False)


File saved: test_predictions_phraseid_sentiment.csv


In [ ]:
from google.colab import files
files.download('test_predictions_phraseid_sentiment.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>